# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
qty_prod_cust = data.groupby(['CustomerID', 'ProductName'])[['Quantity']].sum().reset_index()
qty_prod_cust

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [31]:
p_table = pd.pivot_table(data, values='Quantity', index=['ProductName'], columns=['CustomerID'], aggfunc=sum).fillna(0)
p_table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [32]:
squareform(pdist(p_table, metric='euclidean')),

(array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
         258.0910692 , 278.47800631],
        [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
         279.55500353, 286.80655502],
        [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
         253.01580978, 269.43644891],
        ...,
        [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
         264.14200726, 268.28343221],
        [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
           0.        , 273.15014186],
        [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
         273.15014186,   0.        ]]),)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [33]:
distances = pd.DataFrame(1/(1 + squareform(pdist(p_table.T, 'euclidean'))))
distances

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
1,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
2,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
3,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
4,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
996,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
997,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266
998,0.004167,0.004166,0.004179,0.004175,0.004177,0.004167,0.004168,0.004176,0.004172,0.004161,...,0.003311,0.003162,0.003255,0.003406,0.003172,0.003255,0.003345,0.003277,1.000000,0.003288


In [34]:
dist = pd.DataFrame(1/(1 + squareform(pdist(p_table.T, 'euclidean'))),columns=p_table.columns, index=p_table.columns)
similarities = pd.DataFrame(dist[33].sort_values(ascending=False)[1:6]).reset_index()
similarities

,CustomerID,33
0,264,0.087047
1,3535,0.087047
2,3317,0.087047
3,2503,0.085983
4,3305,0.085638


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [40]:
group = data.groupby(by= ['CustomerID','ProductName'])[['Quantity']].sum().reset_index()
mask = group['CustomerID'].isin([264,3535,3317,2503,3305])
top5 = group[mask]
top5

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [41]:
top5_product_sum = top5.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
top5_product_sum

ProductName
Butter - Unsalted                3
Soup - Campbells Bean Medley     3
Towels - Paper / Kraft           3
Wine - Ej Gallo Sierra Valley    3
Pepper - Black, Whole            3
                                ..
Pepper - White, Ground           1
Pepper - Paprika, Hungarian      1
Pate - Cognac                    1
Pastry - Choclate Baked          1
Anchovy Paste - 56 G Tube        1
Name: Quantity, Length: 218, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [43]:
df_7 = pd.merge(left=top5_product_sum, right=p_table, on='ProductName')
df_7.loc[(df_7[33] == 0),:].sort_values(by='Quantity', ascending=False)['Quantity'][0:5]

ProductName
Butter - Unsalted                3
Wine - Ej Gallo Sierra Valley    3
Wine - Blue Nun Qualitatswein    3
Soup - Campbells Bean Medley     3
Bacardi Breezer - Tropical       2
Name: Quantity, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [45]:
recommendations = {}
IDs = [33, 412, 97793, 97900, 97928, 98069, 98159]

for ID in IDs:
    distances1 = pd.DataFrame(1/(1 + squareform(pdist(p_table.T, 'euclidean'))),columns=p_table.columns, index=p_table.columns)
    similarities = pd.DataFrame(distances1[ID].sort_values(ascending=False)[1:6]).reset_index()

    all_products = data.groupby(['ProductName'])[['Quantity']].sum()
    client = data.query(f'CustomerID == {ID}').groupby(by=['ProductName'])[['Quantity']].sum()
    products = all_products.merge(client, how='left', left_on='ProductName', right_on='ProductName')
        
    filtro  = products['Quantity_y'].isnull() == True
    product = products[filtro].sort_values(by= 'Quantity_x', ascending=False)[1:6].reset_index()

    recommendations.update(product['ProductName'])
    
    print(recommendations)

{0: 'Flavouring - Orange', 1: 'Fenngreek Seed', 2: 'Wine - White Cab Sauv.on', 3: 'Beef - Montreal Smoked Brisket', 4: 'Oil - Shortening - All - Purpose'}
{0: 'Wine - White Cab Sauv.on', 1: 'Bandage - Flexible Neon', 2: 'Beef - Montreal Smoked Brisket', 3: 'Oil - Shortening - All - Purpose', 4: 'Snapple Lemon Tea'}
{0: 'Flavouring - Orange', 1: 'Fenngreek Seed', 2: 'Wine - White Cab Sauv.on', 3: 'Bandage - Flexible Neon', 4: 'Oil - Shortening - All - Purpose'}
{0: 'Flavouring - Orange', 1: 'Fenngreek Seed', 2: 'Wine - White Cab Sauv.on', 3: 'Bandage - Flexible Neon', 4: 'Beef - Montreal Smoked Brisket'}
{0: 'Flavouring - Orange', 1: 'Fenngreek Seed', 2: 'Wine - White Cab Sauv.on', 3: 'Bandage - Flexible Neon', 4: 'Beef - Montreal Smoked Brisket'}
{0: 'Fenngreek Seed', 1: 'Wine - White Cab Sauv.on', 2: 'Bandage - Flexible Neon', 3: 'Beef - Montreal Smoked Brisket', 4: 'French Pastry - Mini Chocolate'}
{0: 'Fenngreek Seed', 1: 'Wine - White Cab Sauv.on', 2: 'Bandage - Flexible Neon', 3: 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [65]:
recommendations = {}
IDs = [33, 412, 97793, 97900, 97928, 98069, 98159]
lista=[]
for ID in IDs:
    distances1 = pd.DataFrame(1/(1 + squareform(pdist(p_table.T, 'euclidean'))),columns=p_table.columns, index=p_table.columns)
    similarities = pd.DataFrame(distances1[ID].sort_values(ascending=False)[1:6]).reset_index()

    all_products = data.groupby(['ProductName'])[['Quantity']].sum()
    client = data.query(f'CustomerID == {ID}').groupby(by=['ProductName'])[['Quantity']].sum()
    products = all_products.merge(client, how='left', left_on='ProductName', right_on='ProductName')
        
    filtro  = products['Quantity_y'].isnull() == True
    product = products[filtro].sort_values(by= 'Quantity_x', ascending=False)[1:6].reset_index()

    recommendations.update(product['ProductName'])
    
    obj = {'ID': ID, 'Prod_1': [recommendations[0]], 'Prod_2': [recommendations[1]], 
           'Prod_3': [recommendations[2]],'Prod_4': [recommendations[3]], 'Prod_5': [recommendations[4]]}
    df = df.append(pd.DataFrame(data=obj, index=[ID])).drop_duplicates().dropna().sort_values(by='ID')

AttributeError: 'NoneType' object has no attribute 'drop_duplicates'

In [64]:
df

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [61]:
recommendations = {}
IDs = [33, 412, 97793, 97900, 97928, 98069, 98159]
lista=[]
for ID in IDs:
    distances1 = pd.DataFrame(1/(1 + squareform(pdist(p_table.T, 'correlation'))),columns=p_table.columns, index=p_table.columns)
    similarities = pd.DataFrame(distances1[ID].sort_values(ascending=False)[1:6]).reset_index()

    all_products = data.groupby(['ProductName'])[['Quantity']].sum()
    client = data.query(f'CustomerID == {ID}').groupby(by=['ProductName'])[['Quantity']].sum()
    products = all_products.merge(client, how='left', left_on='ProductName', right_on='ProductName')
        
    filtro  = products['Quantity_y'].isnull() == True
    product = products[filtro].sort_values(by= 'Quantity_x', ascending=False)[1:6].reset_index()

    recommendations.update(product['ProductName'])
    
    obj = {'ID': ID, 'Prod_1': [recommendations[0]], 'Prod_2': [recommendations[1]], 
           'Prod_3': [recommendations[2]],'Prod_4': [recommendations[3]], 'Prod_5': [recommendations[4]]}
    df = df.append(pd.DataFrame(data=obj, index=[ID])).drop_duplicates().dropna().sort_values(by='ID')

AttributeError: 'NoneType' object has no attribute 'append'